In [38]:
import pandas as pd
from sqlalchemy import create_engine
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
import numpy as np
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings("ignore")

In [2]:
def forecast_yoy_growth_auto(df, col, forecast_steps=12, seasonal=True):
    """
    최적 SARIMAX 파라미터를 자동 탐색하여 수출입 증가율 예측.
    월별 데이터가 72개월 미만일 경우 예측을 수행하지 않습니다.

    Parameters:
        df (pd.DataFrame): 'period' (datetime), 예측 대상 컬럼 포함
        col (str): 예측 컬럼 (예: 'expDlr_yoy')
        forecast_steps (int): 예측 기간 (월 기준 12, 분기 기준 4)
        seasonal (bool): 계절성 사용 여부
    """
    df = df[['period', col]].dropna().copy()
    df = df.set_index('period').sort_index()
    ts = df[col]

    # ✅ 데이터 길이 검사
    if forecast_steps > 4 and len(ts) < 72:
        print("⚠️ 월 데이터가 6년치(72개월) 미만입니다. 충분한 데이터 기간 기준에 미달합니다.")
        return None
    elif forecast_steps <= 4 and len(ts) < 16:
        print("⚠️ 분기 데이터가 4년치(16분기) 미만입니다. 충분한 데이터 기간 기준에 미달합니다.")
        return None

    # SARIMA 파라미터 범위
    p = d = q = range(0, 2)
    pdq = [(x, y, z) for x in p for y in d for z in q]
    seasonal_pdq = [(x, y, z) for x in p for y in d for z in q]
    seasonal_period = 12 if forecast_steps > 4 else 4

    best_model, best_order, best_seasonal_order = sarimax_grid_search(ts, pdq, seasonal_pdq, seasonal_period)

    # 예측
    future_index = pd.date_range(
        start=ts.index[-1] + pd.offsets.DateOffset(months=1 if seasonal_period == 12 else 3),
        periods=forecast_steps,
        freq='M' if seasonal_period == 12 else 'Q'
    )

    forecast = best_model.forecast(steps=forecast_steps)
    forecast.index = future_index

    # 시각화
    plt.figure(figsize=(12, 6))
    plt.plot(ts.index, ts, label='Actual')
    plt.plot(forecast.index, forecast, label='Forecast', linestyle='--', color='red')
    plt.title(f"{col} 예측 (Auto SARIMAX)")
    plt.ylabel('YoY Growth')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [42]:
# ✅ 수정된 DB 구조에 맞춘 데이터 로딩 함수
def load_monthly_trade_data_from_db():
    # host = 'hystox74.synology.me'
    host = '192.168.0.230'
    port = 3307
    user = 'stox7412'
    password = 'Apt106503!~'
    database = 'investar'
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')
    query = "SELECT * FROM korea_monthly_trade_data"
    df = pd.read_sql(query, con=engine)
    return df


# ✅ 수정된 long format 기반 SARIMA 예측 함수
def forecast_yoy_growth_auto_with_ci(df, col, hs_code, forecast_steps):
    warnings.filterwarnings("ignore")
    df_hs = df[(df['root_hs_code'] == hs_code) & (df['indicator'] == col)].copy()

    if df_hs.empty:
        print(f"❌ {hs_code} | '{col}'에 해당하는 데이터가 없습니다.")
        return None, None

    df_hs['date'] = pd.to_datetime(df_hs['date'])
    df_hs = df_hs.set_index('date').sort_index()
    ts = df_hs['value'].astype(float).dropna()

    if forecast_steps > 4 and len(ts) < 72:
        print(f"⚠️ {hs_code}: 월 데이터가 6년치 미만입니다.")
        return None, None

    best_aic = np.inf
    best_order = None
    best_seasonal_order = None
    best_model = None

    p = d = q = range(0, 2)
    P = D = Q = range(0, 2)
    m = 12

    for order in [(p_, d_, q_) for p_ in p for d_ in d for q_ in q]:
        for seasonal_order in [(P_, D_, Q_, m) for P_ in P for D_ in D for Q_ in Q]:
            try:
                model = SARIMAX(ts,
                                order=order,
                                seasonal_order=seasonal_order,
                                enforce_stationarity=False,
                                enforce_invertibility=False)
                result = model.fit(disp=False)
                if result.aic < best_aic:
                    best_aic = result.aic
                    best_order = order
                    best_seasonal_order = seasonal_order
                    best_model = result
            except:
                continue

    if best_model is None:
        print(f"❌ {hs_code}: SARIMAX 모델 최적화 실패")
        return None, None

    forecast_result = best_model.get_forecast(steps=forecast_steps)
    pred_mean = forecast_result.predicted_mean
    conf_int = forecast_result.conf_int()
    future_index = pd.date_range(start=ts.index[-1] + pd.DateOffset(months=1), periods=forecast_steps, freq='M')

    forecast_df = pd.DataFrame({
        col + '_forecast': pred_mean.values,
        col + '_lower': conf_int.iloc[:, 0].values,
        col + '_upper': conf_int.iloc[:, 1].values
    }, index=future_index)

    print(f"✅ {hs_code}: 최적 모델 order={best_order}, seasonal_order={best_seasonal_order}, AIC={best_aic:.2f}")
    return forecast_df, df_hs

# ✅ forecast 시계열 결합 함수 (기존과 동일)
def build_forecasted_series(base_df, forecast_df, col_name):
    full_df = pd.concat([base_df[['value']], forecast_df[[col_name + '_forecast']]], axis=0)
    full_df[col_name + '_forecast'] = pd.to_numeric(full_df[col_name + '_forecast'], errors='coerce')
    full_df[col_name + '_forecast'] = full_df[col_name + '_forecast'].replace([np.inf, -np.inf], np.nan)
    full_df[col_name + '_estimated'] = full_df[col_name + '_forecast'].shift(1) * (1 + full_df[col_name + '_forecast'])
    return full_df

# ✅ 전체 파이프라인 실행 함수
def run_forecasting_pipeline(hs_code_list, max_hs_count=3, forecast_steps = 12):
    export_m_with_yoy = load_monthly_trade_data_from_db()
    hs_code_list = hs_code_list[:max_hs_count]
    result_list = []

    for hs_code in hs_code_list:
        print(f"📈 {hs_code} 예측 중...")
        for target_col in ['expDlr_yoy', 'impDlr_yoy']:
            forecast_df, base_df = forecast_yoy_growth_auto_with_ci(export_m_with_yoy, target_col, hs_code, forecast_steps)

            if forecast_df is None or forecast_df.empty:
                print(f"⚠️ 예측 실패 또는 결과 없음 | hs_code={hs_code}, col={target_col}")
                continue

            full_df = build_forecasted_series(base_df, forecast_df, target_col)
            forecast_only = forecast_df.copy()
            forecast_only['hs_code'] = hs_code
            forecast_only['target_col'] = target_col
            forecast_only['date'] = forecast_only.index
            forecast_only = forecast_only.reset_index(drop=True)

            result_list.append(forecast_only)

    if not result_list:
        print("❌ 모든 예측이 실패하여 결과가 없습니다.")
        return pd.DataFrame()

    final_df = pd.concat(result_list, ignore_index=True)
    return final_df

def merge_forecast_with_actuals(forecast_df, trade_data_df):
    import pandas as pd

    # 📌 1. 복사 및 컬럼명 정리
    forecast_df = forecast_df.copy()
    forecast_df = forecast_df.rename(columns={'hs_code': 'root_hs_code'})  # 혹시 모를 컬럼명 정리

    # 'period' → 'date'로 통일
    if 'period' in forecast_df.columns:
        forecast_df = forecast_df.rename(columns={'period': 'date'})

    forecast_df['date'] = pd.to_datetime(forecast_df['date'])
    trade_data_df['date'] = pd.to_datetime(trade_data_df['date'])

    # 예측 컬럼 확인
    forecast_cols = ['expDlr_yoy_forecast', 'impDlr_yoy_forecast']
    present_forecast_cols = [col for col in forecast_cols if col in forecast_df.columns]
    if not present_forecast_cols:
        raise KeyError("❌ forecast_df에 'expDlr_yoy_forecast' 또는 'impDlr_yoy_forecast' 컬럼이 없습니다.")

    # 📌 2. 무역 데이터 → wide 포맷으로 변환
    trade_pivot = trade_data_df.pivot_table(
        index=['date', 'root_hs_code'],
        columns='indicator',
        values='value',
        aggfunc='last'
    ).reset_index()

    # 📌 3. 예측 데이터 선택
    forecast_selected = forecast_df[['date', 'root_hs_code'] + present_forecast_cols].drop_duplicates()

    # 📌 4. 병합
    combined_df = pd.merge(
        trade_pivot,
        forecast_selected,
        on=['date', 'root_hs_code'],
        how='outer'
    ).sort_values(['root_hs_code', 'date']).reset_index(drop=True)

    # 📌 5. YoY 보정
    combined_df['final_expDlr_yoy'] = combined_df['expDlr_yoy']
    if 'expDlr_yoy_forecast' in combined_df.columns:
        combined_df.loc[combined_df['final_expDlr_yoy'].isna(), 'final_expDlr_yoy'] = combined_df['expDlr_yoy_forecast']

    combined_df['final_impDlr_yoy'] = combined_df['impDlr_yoy']
    if 'impDlr_yoy_forecast' in combined_df.columns:
        combined_df.loc[combined_df['final_impDlr_yoy'].isna(), 'final_impDlr_yoy'] = combined_df['impDlr_yoy_forecast']

    # 📌 6. 자료형 정리
    combined_df['final_expDlr_yoy'] = pd.to_numeric(combined_df['final_expDlr_yoy'], errors='coerce')
    combined_df['final_impDlr_yoy'] = pd.to_numeric(combined_df['final_impDlr_yoy'], errors='coerce')

    # 📌 7. 12개월 전 실적
    combined_df['expDlr_lag12'] = combined_df.groupby('root_hs_code')['expDlr'].shift(12)
    combined_df['impDlr_lag12'] = combined_df.groupby('root_hs_code')['impDlr'].shift(12)

    # 📌 8. 12개월 후 예측
    combined_df['expDlr_forecast_12m'] = combined_df['expDlr_lag12'] * (1 + combined_df['final_expDlr_yoy'])
    combined_df['impDlr_forecast_12m'] = combined_df['impDlr_lag12'] * (1 + combined_df['final_impDlr_yoy'])

    return combined_df

def pivot_forecast_df(forecast_df):
    # 피벗하여 target_col 기준으로 예측값을 컬럼화
    forecast_df = forecast_df.rename(columns={'hs_code': 'root_hs_code'})
    forecast_df['period'] = pd.to_datetime(forecast_df['period'])

    forecast_pivot = forecast_df.pivot_table(index=['period', 'root_hs_code'],
                                             columns='target_col',
                                             values='yoy_forecast').reset_index()

    # 컬럼 이름 정리
    forecast_pivot.columns.name = None
    forecast_pivot = forecast_pivot.rename(columns={
        'expDlr_yoy': 'expDlr_yoy_forecast',
        'impDlr_yoy': 'impDlr_yoy_forecast'
    })
    return forecast_pivot

def upload_forecast_to_db(df, db_info, table_name='korea_trade_forecast_by_month'):
    """
    예측 결과 데이터를 MySQL(MariaDB)에 업로드하는 함수 (중복 시 대체)

    Parameters:
    - df (pd.DataFrame): 업로드할 데이터프레임
    - db_info (dict): DB 연결 정보. keys = ['host', 'port', 'user', 'password', 'database']
    - table_name (str): 업로드할 테이블 이름
    """
    try:
        # ✅ SQLAlchemy 엔진 생성
        engine = create_engine(
            f"mysql+pymysql://{db_info['user']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['database']}"
        )
        conn = engine.raw_connection()
        cursor = conn.cursor()

        # ✅ 테이블 생성 (없을 경우)
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            date DATE,
            root_hs_code VARCHAR(20),
            final_expDlr_yoy FLOAT,
            expDlr_forecast_12m FLOAT,
            PRIMARY KEY (date, root_hs_code)
        );
        """
        cursor.execute(create_table_query)

        # ✅ NaN 및 NaT, inf → None 변환 (모든 타입 안전하게 처리)
        df = df.astype(object).where(pd.notnull(df), None)

        # ✅ REPLACE INTO
        insert_query = f"""
        REPLACE INTO {table_name} (date, root_hs_code, final_expDlr_yoy, expDlr_forecast_12m)
        VALUES (%s, %s, %s, %s)
        """
        data = df[['date', 'root_hs_code', 'final_expDlr_yoy', 'expDlr_forecast_12m']].values.tolist()
        cursor.executemany(insert_query, data)

        conn.commit()
        cursor.close()
        conn.close()

        print(f"✅ 데이터가 {table_name} 테이블에 성공적으로 업로드되었습니다. (중복 시 자동 갱신)")

    except Exception as e:
        print(f"❌ 업로드 실패: {e}")

In [10]:
trade_data_df =  load_monthly_trade_data_from_db()
hs_cd_list = trade_data_df['root_hs_code'].unique().tolist()
# hs_cd_list = ['854232']
forecast_by_month = run_forecasting_pipeline(hs_code_list=hs_cd_list, max_hs_count=len(hs_cd_list), forecast_steps=13)

📈 121120 예측 중...
✅ 121120: 최적 모델 order=(1, 0, 1), seasonal_order=(0, 0, 1, 12), AIC=386.29
✅ 121120: 최적 모델 order=(0, 1, 1), seasonal_order=(0, 1, 1, 12), AIC=2154.31
📈 151550 예측 중...
✅ 151550: 최적 모델 order=(0, 1, 1), seasonal_order=(0, 1, 1, 12), AIC=1247.31
✅ 151550: 최적 모델 order=(0, 1, 1), seasonal_order=(1, 1, 1, 12), AIC=1279.98
📈 151590 예측 중...
✅ 151590: 최적 모델 order=(1, 0, 1), seasonal_order=(1, 1, 1, 12), AIC=639.47
✅ 151590: 최적 모델 order=(0, 1, 1), seasonal_order=(0, 0, 1, 12), AIC=5.70
📈 170199 예측 중...
✅ 170199: 최적 모델 order=(1, 0, 1), seasonal_order=(0, 0, 1, 12), AIC=-2.57
✅ 170199: 최적 모델 order=(1, 1, 1), seasonal_order=(0, 0, 1, 12), AIC=548.22
📈 190230 예측 중...
✅ 190230: 최적 모델 order=(0, 1, 1), seasonal_order=(1, 0, 1, 12), AIC=-238.99
✅ 190230: 최적 모델 order=(0, 1, 1), seasonal_order=(0, 0, 1, 12), AIC=131.45
📈 190590 예측 중...
✅ 190590: 최적 모델 order=(1, 1, 1), seasonal_order=(0, 0, 1, 12), AIC=-275.23
✅ 190590: 최적 모델 order=(1, 0, 1), seasonal_order=(0, 0, 1, 12), AIC=-228.60
📈 20059

In [12]:
# forecast_by_month[forecast_by_month['target_col'] == 'expDlr_yoy'].tail(20)

In [25]:
# 예측 실행 결과 (from run_forecasting_pipeline)
forecast_df = forecast_by_month

# 'expDlr_yoy_forecast' → 'yoy_forecast'로 정리
if 'expDlr_yoy_forecast' in forecast_df.columns:
    forecast_df = forecast_df.rename(columns={'expDlr_yoy_forecast': 'yoy_forecast'})
elif 'impDlr_yoy_forecast' in forecast_df.columns:
    forecast_df = forecast_df.rename(columns={'impDlr_yoy_forecast': 'yoy_forecast'})

# DB에서 데이터 불러오기
trade_data_df = load_monthly_trade_data_from_db()

# 🧩 Step 1: forecast_df 피벗
forecast_df_pivoted = pivot_forecast_df(forecast_df)

# 🧩 Step 2: merge 실행'
final_combined_df = merge_forecast_with_actuals(forecast_df_pivoted, trade_data_df)

# hs_code_list = ['854232', '854231']
# hs_code_list
forecast_result_df = final_combined_df[final_combined_df['root_hs_code'].isin(hs_cd_list)]
forecast_result_data = forecast_result_df[['date', 'root_hs_code', 'final_expDlr_yoy', 'expDlr_forecast_12m']]

In [43]:
db_info = {
    'host' : '192.168.0.230',
    'port' : 3307,
    'user' : 'stox7412',
    'password' : 'Apt106503!~',
    'database' : 'investar'
}

upload_forecast_to_db(forecast_result_data, db_info)

✅ 데이터가 korea_trade_forecast_by_month 테이블에 성공적으로 업로드되었습니다. (중복 시 자동 갱신)


### 예측 정확성 측정을 위해 예측치만 별도로 DB에 보관한다

In [48]:
from datetime import datetime

# 📌 2. 전처리
today_date = datetime.today().strftime('%Y-%m-%d')

# 'period' 제거
if 'period' in forecast_by_month.columns:
    df = forecast_by_month.drop(columns=['period'])

# 'hs_code' → 'root_hs_code'로 변경
df = forecast_by_month.rename(columns={'hs_code': 'root_hs_code'})

# forecast_date 추가
df['forecast_date'] = today_date

# SQLAlchemy 엔진 생성
engine = create_engine(
    f"mysql+pymysql://{db_info['user']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['database']}"
)
# 📌 4. DB 업로드
table_name = 'korea_trade_data_forecast_record'
df.to_sql(name=table_name, con=engine, if_exists='append', index=False)

print(f"✅ {table_name} 테이블에 {len(df)}건 업로드 완료 (날짜: {today_date})")

✅ korea_trade_data_forecast_record 테이블에 13871건 업로드 완료 (날짜: 2025-06-12)
